<a href="https://colab.research.google.com/github/Yunho-seo/education_center4/blob/main/08_pymysql_stored.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00


In [3]:
import pymysql
con = pymysql.connect(host='34.64.86.80', port=3306, user='root', passwd='acorn1234',
                      db='sungjuk', charset='utf8', autocommit=True)
try:
  with con.cursor() as cur:
    cur.execute('select * from student where bunho in (1, 2, 3)')
    rows = cur.fetchall()
    for row in rows:
      print(f'{row[0]:<4} {row[1]:>10} {row[2]:>10} {row[3]:>10} {row[4]:>10}')
    print(f'쿼리가 영향을 미친 것은 {cur.rowcount} 개의 행')
finally:
  con.close()

1           김만덕         80         99         60
2           고려인        100         99         99
3           천공인        100         81        100
쿼리가 영향을 미친 것은 3 개의 행


In [4]:
import pymysql

con = pymysql.connect(host='34.64.86.80', port=3306, user='root', passwd='acorn1234',
                      db='sungjuk', charset='utf8', autocommit=True)

person = ('typoon', 100, 100, 100, 'CH00000001')

try:
  with con.cursor() as cur:
    cur.execute("""insert into student(name, kor, mat, eng, schoolcode)
    values (%s, %s, %s, %s, %s)""",  # 송수신은 문자열 형태
        (person[0], person[1], person[2], person[3], person[4]))
    con.commit()  
    if(cur.rowcount):  # 영향을 미친 열이 있으면 True
      print('새로운 학생이 등록되었습니다.')
finally:
  con.close()

# 환각(할루시네이션) : 사실이 아닌 문자나 이미지를 마치 맞는 답인 것처럼 내놓은 현상.

새로운 학생이 등록되었습니다.


In [10]:
# 문제
# 1. 김만덕 => '오환각' 으로 이름 바꾸기
# 2. 입력된 'typoon' 삭제하기 
import pymysql

con = pymysql.connect(host='34.64.86.80', port=3306, user='root', passwd='acorn1234',
                      db='sungjuk', charset='utf8', autocommit=True)

person = ('김만덕', '오환각')
person_1 = ('typoon')
try:
  with con.cursor() as cur:
    cur.execute("""update student set name = %s where name = %s""", (person[1], person[0]))
    cur.execute("""delete from student where name = %s """, person_1)
  con.commit()
  if(cur.rowcount):
    print('수정, 삭제가 완료되었습니다.')
finally:
  con.close()

In [18]:
# 저장 프로시저 (stored procedure call)
# student_select 프로시저 

con = pymysql.connect(host='34.64.86.80', port=3306, user='root', passwd='acorn1234',
                      db='sungjuk', charset='utf8', cursorclass=pymysql.cursors.DictCursor, autocommit=True)  # Dict

try:
  with con.cursor() as cur:
    cur.callproc('student_select')
    if(cur.rowcount):
      print(cur.fetchall())
    cur.callproc('student_select')
    if(cur.rowcount):  # 데이터가 있으면
      data = cur.fetchall()
      for row in data:
        # print(row[0], row[1])
        print(row["bunho"], row["name"], row["kor"])
finally:
  con.close()

[{'bunho': 1, 'name': '오환각', 'kor': '80', 'mat': '99', 'eng': '60', 'total': 239, 'average': 79.67, 'grade': 'C', 'schoolcode': 'CH00000001'}, {'bunho': 2, 'name': '고려인', 'kor': '100', 'mat': '99', 'eng': '99', 'total': 298, 'average': 99.33, 'grade': 'A', 'schoolcode': 'CH00000001'}, {'bunho': 3, 'name': '천공인', 'kor': '100', 'mat': '81', 'eng': '100', 'total': 281, 'average': 93.67, 'grade': 'A', 'schoolcode': 'SE00000001'}, {'bunho': 4, 'name': '천공이', 'kor': '81', 'mat': '81', 'eng': '81', 'total': 243, 'average': 81.0, 'grade': 'B', 'schoolcode': 'SE00000001'}, {'bunho': 5, 'name': '천공삼', 'kor': '91', 'mat': '100', 'eng': '100', 'total': 291, 'average': 97.0, 'grade': 'A', 'schoolcode': 'CH00000001'}, {'bunho': 6, 'name': '천공사', 'kor': '100', 'mat': '100', 'eng': '100', 'total': 300, 'average': 100.0, 'grade': 'A', 'schoolcode': 'CH00000001'}, {'bunho': 7, 'name': '종로구', 'kor': '100', 'mat': '81', 'eng': '71', 'total': 252, 'average': 84.0, 'grade': 'B', 'schoolcode': 'IC00000001'},

In [30]:
# student_insert 프로시저 

con = pymysql.connect(host='34.64.86.80', port=3306, user='root', passwd='acorn1234',
                      db='sungjuk', charset='utf8', cursorclass=pymysql.cursors.DictCursor, autocommit=True)

data = ('미래융합교육원', 90, 80, 100, 'CH00000001', 0)
try:
  with con.cursor() as cur:
    cur.callproc('student_insert', data)
    cur.execute('select @_student_insert_5')
    con.commit()
    result = cur.fetchone()
    print(result['@_student_insert_5'])  # 0이면 성공
finally:
  con.close()

0


In [23]:
# student_update 프로시저 

conn = pymysql.connect(host='34.64.86.80', port=3306, user='root', passwd='acorn1234',
                      db='sungjuk', charset='utf8', cursorclass=pymysql.cursors.DictCursor, autocommit=True)

data = (1, '왕만덕', 100, 90, 80, 0)
try:
  with conn.cursor() as cur:
    cur.callproc('student_update', data)
    cur.execute('select @_student_update_5')
    conn.commit()
    result = cur.fetchone()
    print(result['@_student_update_5'])
    if result['@_student_update_5'] == 2:
      print('수정할 이름이 없습니다.')
    elif result['@_student_update_5'] == -1:
      print('에러가 발생하였습니다.')
    elif result['@_student_update_5'] == 0:
      print('수정 성공하였습니다.')
    else:
      print('알 수 없음')
finally:
  conn.close()

0
수정 성공하였습니다.


In [31]:
# 문제
# 미래융합교육원 삭제 (stored procedure call을 이용하여)

conn = pymysql.connect(host='34.64.86.80', port=3306, user='root', passwd='acorn1234',
                      db='sungjuk', charset='utf8', cursorclass=pymysql.cursors.DictCursor, autocommit=True)

data = ('미래융합교육원', 0)
try:
  with conn.cursor() as cur:
    cur.callproc('student_delete', data)
    cur.execute('select @_student_delete_1')  # _1 하는 이유는 data의 result(여기서는 [1])에 접근하기 위해.
    conn.commit()
    result = cur.fetchone()
    print(result['@_student_delete_1'])
    if result['@_student_delete_1'] == 2:
      print('삭제할 이름이 없습니다.')
    elif result['@_student_delete_1'] == -1:
      print('에러가 발생하였습니다.')
    elif result['@_student_delete_1'] == 0:
      print('삭제 성공하였습니다.')
    else:
      print('알 수 없음')
finally:
  conn.close()

0
삭제 성공하였습니다.


In [35]:
conn = pymysql.connect(host='34.64.86.80', port=3306, user='root', passwd='acorn1234',
                      db='zoo', charset='utf8', cursorclass=pymysql.cursors.DictCursor, autocommit=True)

cur = conn.cursor()

cur.execute("""CREATE TABLE `pet` (
	`id` INT(11) NOT NULL AUTO_INCREMENT,
	`name` VARCHAR(20) NULL DEFAULT NULL,
	`owner` VARCHAR(20) NULL DEFAULT NULL,
	`species` VARCHAR(20) NULL DEFAULT NULL,
	`sex` CHAR(1) NULL DEFAULT NULL,
	`birth` DATE NULL DEFAULT NULL,
	`death` DATE NULL DEFAULT NULL,
	PRIMARY KEY (`id`)
)
COLLATE='utf8mb4_general_ci'
ENGINE=InnoDB
;""")

OperationalError: ignored

In [36]:
print(cur.description)
conn.close()

None


In [41]:
conn = pymysql.connect(host='34.64.86.80', port=3306, user='root', passwd='acorn1234',
                      db='zoo', charset='utf8', cursorclass=pymysql.cursors.DictCursor, autocommit=True)
cur = conn.cursor()

sql = """INSERT INTO pet (name, owner, species, sex, birth, death)
        VALUES(%s,%s,%s,%s,%s,%s)"""
# cur.execute(sql, (('인천시','민주','cat','f','2001-02-04', None))
# cur.execute(sql, (('대구시','자유','cat','f','2010-03-17', None))
# cur.execute(sql, (('대전시','민주','dog','f','2010-05-13', None))
# cur.execute(sql, (('광주시','민주','dog','m','2015-08-27', None))
# cur.execute(sql, ((('부산시','자유','dog','m','2017-08-31','2018-04-29'))
# conn.commit()

petlist = [('인천시','민주','cat','f','2001-02-04', None),
              ('대구시','자유','cat','f','2010-03-17', None),
              ('대전시','민주','dog','f','2010-05-13', None),
              ('광주시','민주','dog','m','2015-08-27', None),
              ('부산시','자유','dog','m','2017-08-31','2018-04-29')]
cur.executemany(sql, petlist)
conn.commit()
print(cur.description)
print(cur.rowcount)

conn.close()

None
5


In [46]:
conn = pymysql.connect(host='34.64.86.80', port=3306, user='root', passwd='acorn1234',
                      db='zoo', charset='utf8', cursorclass=pymysql.cursors.DictCursor, autocommit=True)

cursor = conn.cursor()
cursor.execute('select * from pet')
print(cursor.rowcount)
cursor.fetchone()
cursor.fetchone()
cursor.fetchone()
cursor.scroll(0, mode='absolute')  # 0번으로 커서를 스크롤 # relative
recs = cursor.fetchmany(2)
for rec in recs:
  print(rec)
recs = cursor.fetchall()  # 커서가 마지막을 가리키고 있기 때문에, 결과가 없다.
for rec in recs:
  print('데이터가 있으면 찍힌다.')
  print(rec)
cursor.close()
conn.close()

5
{'id': 1, 'name': '인천시', 'owner': '민주', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2001, 2, 4), 'death': None}
{'id': 2, 'name': '대구시', 'owner': '자유', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2010, 3, 17), 'death': None}
데이터가 있으면 찍힌다.
{'id': 3, 'name': '대전시', 'owner': '민주', 'species': 'dog', 'sex': 'f', 'birth': datetime.date(2010, 5, 13), 'death': None}
데이터가 있으면 찍힌다.
{'id': 4, 'name': '광주시', 'owner': '민주', 'species': 'dog', 'sex': 'm', 'birth': datetime.date(2015, 8, 27), 'death': None}
데이터가 있으면 찍힌다.
{'id': 5, 'name': '부산시', 'owner': '자유', 'species': 'dog', 'sex': 'm', 'birth': datetime.date(2017, 8, 31), 'death': datetime.date(2018, 4, 29)}


In [ ]:
# 문제
# 1. 모든 pet을 가져오는 프로시저를 만들고 callproc로 데이터 출력
# 2. 다음은 쿼리문으로 직접 실행
#  2.1) 생일이 2015-01-01 이후인 pet 출력
#  2.2) species = 'cat'이고 여성만 출력
#  2.3) 이름과 생일을, 생일로 정렬하여 출력
#  2.4) 사망일이 Null인 데이터 출력
#  2.5) 이름이 대구시인 데이터를 부산시로 변경

In [52]:
conn = pymysql.connect(host='34.64.86.80', port=3306, user='root', passwd='acorn1234',
                      db='zoo', charset='utf8', cursorclass=pymysql.cursors.DictCursor, autocommit=True)

try:
  cursor = conn.cursor()
  cursor.callproc('pet_all')
  cur = cursor.fetchall()  # 저장 프로시저에서 select하는 것은, 모두 결합되어 리턴된다.
  for row in cur:
    print("ID = %s, name = %s" % (row['id'], row['name']))
  cursor.nextset()
  for row in cur:
    print("ID = %s, name = %s" % (row['id'], row['name']))
finally:
  cursor.close()
  conn.close()

ID = 1, name = 인천시
ID = 2, name = 대구시
ID = 3, name = 대전시
ID = 4, name = 광주시
ID = 5, name = 부산시
ID = 1, name = 인천시
ID = 2, name = 대구시
ID = 3, name = 대전시
ID = 4, name = 광주시
ID = 5, name = 부산시


In [56]:
conn = pymysql.connect(host='34.64.86.80', port=3306, user='root', passwd='acorn1234',
                      db='zoo', charset='utf8', cursorclass=pymysql.cursors.DictCursor, autocommit=True)

try:
  cursor = conn.cursor()
  # cursor.execute("select * from pet where birth > '2015-01-01'")
  # cursor.execute("select * from pet where species = 'cat' and sex = 'f'")
  # cursor.execute("select name, birth from pet order by 'birth'")
  # cursor.execute("select name, birth from pet where death is NULL ")
  rows = cursor.fetchall()
  print(type(rows))
  for pets in list(rows):      # [{}, {}, {}]
    pet = list(pets.values())  # keys + values의 결합
    print("ID = %d, Name = %s, 주인 = %s" % (pet[0], pet[1], pet[2]))
finally:
  cursor.close()
  conn.close()

<class 'list'>


IndexError: ignored

In [61]:
conn = pymysql.connect(host='34.64.86.80', port=3306, user='root', passwd='acorn1234',
                      db='zoo', charset='utf8', cursorclass=pymysql.cursors.DictCursor, autocommit=True)

try:
  cursor = conn.cursor()
  sql = 'update pet set name = %s where name = %s'
  cursor.execute(sql, ('대구시', '부산시'))
  conn.commit()
  print('실행완료')
  print(cursor.rowcount)
  cursor.description
finally:
  cursor.close()
  conn.close()

실행완료
2


In [ ]:
# class에 데이터를 입력한다.
# 트리거를 이용하여 합계, 평균, 학점을 자동으로 입력하도록 저장 프로시저를 작성
# 이후, 클래스를 설계
# 1명분의 데이터를 저장(name, kor, mat, eng, total, average, grade)
# 입력 함수, 계산(합계, 평균) 함수, 학점 계산 함수, 출력 함수(__repr__)

# 리스트에 입력 (데이터베이스와 리스트에 있는 내용이 일치해야 한다)
# - maria db 연결 (select, insert, update, delete DB)
# - CRUD를 실행시켜서, class list와 일치되게 관리
# - list CRUD == DB CRUD와 연결한다. (프로그램이 종료되어도 DB에는 데이터가 남아있어야 함)
# Management class
# - 메뉴에 따라 CRUD가 직접 실행

# 메뉴 (함수, class)

In [127]:
import pymysql
class Student:
  def __init__(self):
    self.bunho = 0;
    self.name = ''
    self.kor = 0;
    self.mat = 0;
    self.eng = 0;
    self.total = 0;
    self.average = 0;
    self.grade = ''
  
  def inputData(self):
    print('학생 데이터를 입력하시오')
    self.name = input('이름을 입력하시오 : ')
    self.kor = eval(input('국어점수를 입력하시오 : '))
    self.mat = eval(input('수학점수를 입력하시오 : '))
    self.eng = eval(input('영어점수를 입력하시오 : '))
    self.calc_total_average()
  
  def calc_total_average(self):
    self.total = self.kor + self.mat + self.eng
    self.average = round(self.total/3, 2)
    # self.calcSemGrade()
  
  def calcSemGrade(self):
    if self.average >=90: return 'A'
    elif self.average >=80: return 'B'
    elif self.average >=70: return 'C'
    elif self.average >=60: return 'D'
    else: return 'F'
  
  def __repr__(self):
    return "%5s %7s %8s %8s %8s %8s %9s %3s" % (self.bunho, self.name, self.kor, self.mat, self.eng, self.total, self.average, self.grade)



In [128]:
class maria:
  schoolname = '대전미래융합교육원'
  def __init__(self):
    self.conn = pymysql.connect(host='34.64.86.80', port=3306, user='root', passwd='acorn1234',
                      db='zoo', charset='utf8', #cursorclass=pymysql.cursors.DictCursor, 
                      autocommit=True)
    
    cor = self.conn.cursor()
    cor.execute("show tables")
    tables = cor.fetchall()
    c=[]
    for i in tables:
      if i == ('sungjuk',):
        c.append(i)
    # if len(c) !=0:
    #   sungjuk_table = "select * from sungjuk"
    #   cor.execute(sungjuk_table)
    #   print("테이블이 있습니다. ")
    # else:
    #   cor.execute(""" CREATE TABLE `sungjuk` (
    #                   `bunho` INT(11) NOT NULL AUTO_INCREMENT,
    #                   `name` VARCHAR(20) NULL DEFAULT NULL COLLATE 'utf8mb4_general_ci',
    #                   `kor` INT(11) NULL DEFAULT NULL,
    #                   `mat` INT(11) NULL DEFAULT NULL,
    #                   `eng` INT(11) NULL DEFAULT NULL,
    #                   `total` INT(11) NULL DEFAULT NULL,
    #                   `average` FLOAT NULL DEFAULT NULL,
    #                   `grade` CHAR(1) NULL DEFAULT NULL COLLATE 'utf8mb4_general_ci',
    #                   PRIMARY KEY (`bunho`) USING BTREE
    #                 )
    #                 COLLATE='utf8mb4_general_ci'
    #                 ENGINE=InnoDB
    #                 ; """)
                          
  def __del__(self):
    self.conn.close()
    
  def selectdb(self):
    cur=self.conn.cursor()
    cur.callproc("sungjuk_select")
    if cur.rowcount :
      std = list(cur.fetchall())
    else:
      std = 0
    cur.close()
    return std

  def insertdb(self, name, kor, mat, eng, result):
    cur = self.conn.cursor()
    arg = (name, kor, mat, eng, result)
    cur.callproc("sungjuk_insert", arg)
    cur.close()

  def updatedb(self, in_name, up_name, result):
    cur = self.conn.cursor()
    arg = (in_name, up_name, result)
    cur.callproc("sungjuk_update", arg)
    cur.close();

  def deletedb(self, in_name, result):
    cur = self.conn.cursor()
    arg = (in_name, result)
    cur.callproc("sungjuk_delete", arg)
    cur.close()

In [129]:
from math import *
class Management:
  def __init__(self):
    self.maria = maria();
    
  def M_select(self):
    data = self.maria.selectdb()
    st = []
    if(data):
      for student in data:
        stp = Student()
        stp.bunho = student[0]
        stp.name = student[1]
        stp.kor = student[2]
        stp.mat = student[3]
        stp.eng = student[4]
        # 내가 자체적 추가
        stp.total = student[5]
        stp.average = student[6]
        stp.grade = student[7]
        st.append(stp)
      print(self.maria.schoolname + " 성적 계산표 ")
      print( "%5s %5s %6s %6s %6s %7s %7s %s" % ('번호', '이름', '국어', '수학', '영어', '합계', '평균', '학점') )
      for stud in st:
        print(stud)
    else:
      print("데이터가 없습니다.")

  def M_insert(self):
    stu = Student()
    stu.inputData()
    self.maria.insertdb(stu.name, stu.kor, stu.mat, stu.eng, 0)

  def M_update(self):
    name = input("검색할 이름을 입력하시오 : ")
    replace_name = input("수정할 이름을 입력하시오 : ")
    result = self.maria.updatedb(name, replace_name, 0)

  def M_delete(self):
    name = input("삭제할 이름을 입력하시오 : ")
    self.maria.deletedb(name, 0)

In [130]:
import sys
class Menu():
  def __init__(self):
    self.manage = Management()
  
  def work(self):
    menu = ['전체검색(1)', '입력(2)', '수정(3)', '삭제(4)', '종료(9)']
    while(1):
      sel = input(menu)
      if sel == '1':
        self.manage.M_select()
        continue
      elif sel == '2':
        self.manage.M_insert()
      elif sel == '3':
        self.manage.M_update()
      elif sel == '4':
        self.manage.M_delete()
      elif sel == '9':
        break;
        # sys.exit()
      else:
        print('잘못된 입력입니다. 다시 입력하세요')
        continue

menu = Menu()
menu.work()

['전체검색(1)', '입력(2)', '수정(3)', '삭제(4)', '종료(9)']2
학생 데이터를 입력하시오
이름을 입력하시오 : 이병열열
국어점수를 입력하시오 : 100
수학점수를 입력하시오 : 90
영어점수를 입력하시오 : 100
['전체검색(1)', '입력(2)', '수정(3)', '삭제(4)', '종료(9)']1
대전미래융합교육원 성적 계산표 
   번호    이름     국어     수학     영어      합계      평균 학점
    6    서윤호호      100      100       80      280     93.33   A
    7     서윤호       90       90       90      270      90.0   A
    8     이병열      100      100      100      300     100.0   A
    9    이병열열      100       90      100      290     96.67   A
['전체검색(1)', '입력(2)', '수정(3)', '삭제(4)', '종료(9)']9
